In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

# Visualize US Data

In [ ]:
from geonotebook.wrappers import RasterData

In [ ]:
rd = RasterData("geotrellis://http://localhost:8899/tiles/weld11")

In [ ]:
M.add_layer(rd)

# Subset with a rectangle

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyproj import Proj, transform
from shapely import geometry
from geopyspark.geotrellis.catalog import S3Catalog
from geopyspark.geopycontext import GeoPyContext
from geopyspark.avroregistry import AvroRegistry
import rasterio

from gnb_geotrellis.globalmaptiles import get_extent

In [ ]:
def latLongToWebMercator(lat_long_coords):
    in_proj  = Proj("+init=EPSG:4326")
    out_proj = Proj("+init=EPSG:3857")
    return [transform(in_proj, out_proj, x, y) for (x, y) in lat_long_coords]

In [ ]:
# If sparkcontext already exists use it
try:
    geopysc = GeoPyContext(master='local[*]', appName='data_frame_test')
except ValueError:
    pass

catalog = S3Catalog(geopysc)


In [ ]:
from rasterio.transform import Affine

def transform_from_corner(ulx, uly, dx, dy):
    return Affine.translation(ulx, uly)*Affine.scale(dx, -dy)


# Get Total RDD Extent

In [ ]:
annot = M.layers.annotation.rectangles[0]
extent = get_extent(13, annot)
rdd_polygon = geometry.box(*extent)
geojson = geometry.geo.mapping(rdd_polygon)

In [ ]:
key_type = 'spatial'
value_type = 'singleband'
bucket = "kitware-catalog"
prefix = 'full-catalog'
layer_name = 'weld11'
layer_zoom = 13
polygon = geometry.Polygon(
    latLongToWebMercator(list(annot.exterior.coords)))

(rdd, schema, metadata) = catalog.query(key_type, value_type, bucket, 
                                prefix, layer_name, layer_zoom, polygon)

In [ ]:
ta = rdd.collect()

# Investigate

In [ ]:
def stitch_tilearrays(ta):
    arr = None
    c_arr = None
    prev_col = None

    for key, _ta in sorted(ta, key=lambda x: (x[0]['col'], x[0]['row'])):
        # Build up the column
        if prev_col == None:
            c_arr = _ta['data']
        elif prev_col == key['col']:
            c_arr = np.vstack([c_arr, _ta['data']])
        # We've finished with the column,  add it
        # to the array
        else:
            if arr is None:
                arr = c_arr
            else:
                arr = np.hstack([arr, c_arr])

            c_arr = _ta['data']
        prev_col = key['col']

    return np.hstack([arr, c_arr])

In [ ]:
tilearray = stitch_tilearrays(ta)
print(tilearray.shape)
plt.imshow(tilearray)